In [73]:
import pandas as pd
import numpy as np
# sklearn load_boston no longer works
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

In [74]:
# make the data and target np arrays into a dataframe with appropriate column names
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
data_values = pd.DataFrame(data, columns=column_names)
data_values['MEDV'] = target
data_values.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [75]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler

In [76]:
# declaring the features and target and scaling features
X = data_values[column_names]
y = data_values['MEDV']

scale = StandardScaler()

X = scale.fit_transform(X)

In [77]:
# creating testing and training splits
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)

In [78]:
# helper function that will declare the model with the desired parameters, fit it, and print the mse
def get_mse(learning_rate, regularization):
  sgdr = SGDRegressor(eta0=learning_rate, penalty=regularization)
  sgdr.fit(X_train, y_train)
  y_pred = sgdr.predict(X_test)
  mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
  print(f'Mean squared error (L{regularization[1:]} regularization & {learning_rate} learning rate):', mse)


In [79]:
# we want to test different learning rates (0.001, 0.01, 0.1) with both L1 and L2 regularization
l1 = 'l1'
# L1 regularization with eta = 0.001
get_mse(0.001, l1)
# L1 regularization with eta = 0.01
get_mse(0.01, l1)
# L1 regularization with eta = 0.1
get_mse(0.1, l1)

Mean squared error (L1 regularization & 0.001 learning rate): 17.937788756981092
Mean squared error (L1 regularization & 0.01 learning rate): 18.709361780437174
Mean squared error (L1 regularization & 0.1 learning rate): 23.132904149736003


In [80]:
# now do the above process with l2 regularization
l2 = 'l2'
# L2 regularization with eta = 0.001
get_mse(0.001, l2)
# L2 regularization with eta = 0.01
get_mse(0.01, l2)
# L2 regularization with eta = 0.1
get_mse(0.1, l2)

Mean squared error (L2 regularization & 0.001 learning rate): 17.888770269744644
Mean squared error (L2 regularization & 0.01 learning rate): 19.068210677552486
Mean squared error (L2 regularization & 0.1 learning rate): 23.487403453159374


In [81]:
# helper function that will fit each model 100 times and average the mse over those 100 times to get
# a better idea of which model performs better/worse on average
import statistics

def get_avg_mse(model):
  errors = []
  for i in range(101):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
    errors.append(mse)
  return statistics.mean(errors)

In [82]:
# now each of those regularization/learning rate combos will be run 100 times and the errors of the
# trials will be averaged to determine more accurate error values
model = SGDRegressor(eta0=0.001, penalty='l1')
model.fit(X_train, y_train)
avg_mse = get_avg_mse(model)
print('The average mse after 100 trials for SGD regression with L1 regulzarization and a learning rate of 0.001:', avg_mse)
model = SGDRegressor(eta0=0.01, penalty='l1')
model.fit(X_train, y_train)
avg_mse = get_avg_mse(model)
print('The average mse after 100 trials for SGD regression with L1 regulzarization and a learning rate of 0.01:', avg_mse)
model = SGDRegressor(eta0=0.1, penalty='l1')
model.fit(X_train, y_train)
avg_mse = get_avg_mse(model)
print('The average mse after 100 trials for SGD regression with L1 regulzarization and a learning rate of 0.1:', avg_mse)
model = SGDRegressor(eta0=0.001, penalty='l2')
model.fit(X_train, y_train)
avg_mse = get_avg_mse(model)
print('The average mse after 100 trials for SGD regression with L2 regulzarization and a learning rate of 0.001:', avg_mse)
model = SGDRegressor(eta0=0.01, penalty='l2')
model.fit(X_train, y_train)
avg_mse = get_avg_mse(model)
print('The average mse after 100 trials for SGD regression with L2 regulzarization and a learning rate of 0.01:', avg_mse)
model = SGDRegressor(eta0=0.1, penalty='l2')
model.fit(X_train, y_train)
avg_mse = get_avg_mse(model)
print('The average mse after 100 trials for SGD regression with L2 regulzarization and a learning rate of 0.1:', avg_mse)

The average mse after 100 trials for SGD regression with L1 regulzarization and a learning rate of 0.001: 17.89095872356958
The average mse after 100 trials for SGD regression with L1 regulzarization and a learning rate of 0.01: 18.917294002497243
The average mse after 100 trials for SGD regression with L1 regulzarization and a learning rate of 0.1: 20.94021686498111
The average mse after 100 trials for SGD regression with L2 regulzarization and a learning rate of 0.001: 17.905981946453483
The average mse after 100 trials for SGD regression with L2 regulzarization and a learning rate of 0.01: 19.046102586969038
The average mse after 100 trials for SGD regression with L2 regulzarization and a learning rate of 0.1: 18.096875160137788


In [83]:
# now train AdaBoost Classifier on the data and compare the results
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

In [84]:
# declare decision stumps to use in AdaBoost
stump = DecisionTreeRegressor(max_depth=1)

In [85]:
# declare AdaBoost model
adaboost = AdaBoostRegressor(estimator=stump, n_estimators=5000, learning_rate=0.1)

In [86]:
# get predictions from AdaBoost model and get MSE to compare with linear regression models
adaboost.fit(X_train, y_train)
y_pred = adaboost.predict(X_test)

mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
# using AdaBoost with strict decision stumps (max_depth = 1) performs poorly as the data has too
# many features and the stumps are not able to provide enough information
print('Mean squared error (AdaBoost Regression with decision stumps):', mse)

Mean squared error (AdaBoost Regression with decision stumps): 43.154288284305494


In [87]:
# get the average mse after 100 trials
print('The average mse after 100 trials for AdaBoost regression with decision stumps:', get_avg_mse(adaboost))

The average mse after 100 trials for AdaBoost regression with decision stumps: 43.154288284305494


In [88]:
# try AdaBoost with more complex base estimator
tree = DecisionTreeRegressor(max_depth=4)
adaboost = AdaBoostRegressor(estimator=tree, n_estimators=5000, learning_rate=0.1)

adaboost.fit(X_train, y_train)
y_pred = adaboost.predict(X_test)

mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
# using AdaBoost with strict decision trees allow AdaBoost to gather more information
# and thus make better predictions
print('Mean squared error (AdaBoost Regression with decision trees):', mse)

Mean squared error (AdaBoost Regression with decision trees): 10.828768047451817


In [89]:
# get the average mse after 100 trials
print('The average mse after 100 trials for AdaBoost regression with decision trees:', get_avg_mse(adaboost))

The average mse after 100 trials for AdaBoost regression with decision trees: 10.828768047451817


In [90]:
# now try training a neural network to perform the regression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, BatchNormalization
from keras import regularizers

In [91]:
# define the neural network and add an input + hidden layers
nn = Sequential()
nn.add(InputLayer((X_train.shape[1],)))
# add a lay with regularization
nn.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# Batch normalization to normalize the outputs from each hidden layer before passing them along
# Batch normalization ensures that the data distribution remains consistent
nn.add(BatchNormalization())
nn.add(Dense(8, activation='relu'))
nn.add(BatchNormalization())
# linear activation function for output layer in regression
nn.add(Dense(1, activation='linear'))

In [92]:
# compile the neural network and fit it to the data
nn.compile(optimizer='adam', loss='mse', metrics=['mse'])

nn.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=8, verbose=1)

Epoch 1/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 608.1345 - mse: 606.8460 - val_loss: 613.0177 - val_mse: 611.7343
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 603.2843 - mse: 602.0015 - val_loss: 592.3120 - val_mse: 591.0312
Epoch 3/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 586.6337 - mse: 585.3538 - val_loss: 570.4092 - val_mse: 569.1320
Epoch 4/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 549.9066 - mse: 548.6298 - val_loss: 546.6674 - val_mse: 545.3947
Epoch 5/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 518.5726 - mse: 517.3018 - val_loss: 524.4647 - val_mse: 523.1982
Epoch 6/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 526.6467 - mse: 525.3805 - val_loss: 496.3952 - val_mse: 495.1333
Epoch 7/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 527.5766 - mse: 526.3162 - val_loss: 473.7166 - val_mse: 472.4616
Epoch 8/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 438.8155 - mse: 437.5622 - val_loss: 454.7588 - val_mse: 453.5105

In [93]:
# now get predictions on the test data and mse
y_pred = nn.predict(X_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [94]:
# it is clear that the neural network performed the best but also required the most computational resources
# the neural network was able to capture nuances and nonlinearity in the data that the linear regression
# may not have been able to. However, AdaBoost with decision trees performed better and was both easier to
# write and computationally more efficient than a neural network
mse = mean_squared_error(y_pred=y_pred, y_true=y_test)
print('Mean squared error (neural network):', mse)

Mean squared error (neural network): 12.02974566243046


In [95]:
# get the average mse after 100 trials
print('The average mse after 100 trials for a neural network:', get_avg_mse(nn))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
4/4 ━━━━━━━━